In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf
import numpy as np

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'    

In [3]:
if tf.test.gpu_device_name():
    print('GPU found', tf.test.gpu_device_name())
else:
    print("No GPU found")

No GPU found


In [ ]:
model = VGG16()
model.summary()

In [4]:
from vgg import VGG

In [57]:
tf.reset_default_graph()

In [58]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y_true = tf.placeholder(tf.float32, shape=[None])
keep_prob = tf.placeholder(tf.float32)

model = VGG(x, keep_prob, 10)


In [7]:
loss = model.loss(y_true)
trainer = model.trainer(loss)

W0212 21:44:28.988582 140528210544448 deprecation.py:323] From /home/harry/DeepLearning/StyleTransfer_VGG/tf/vgg.py:149: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0212 21:44:29.085319 140528210544448 deprecation_wrapper.py:119] From /home/harry/DeepLearning/StyleTransfer_VGG/tf/vgg.py:153: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.



NameError: name 'lr' is not defined

In [20]:
from data_utils import get_CIFAR10_data

In [12]:
data = get_CIFAR10_data()
for k, v in data.items():
    print('%s: ' % k, v.shape)



X_train:  (49000, 3, 32, 32)
y_train:  (49000,)
X_val:  (1000, 3, 32, 32)
y_val:  (1000,)
X_test:  (1000, 3, 32, 32)
y_test:  (1000,)


In [59]:
def one_hot(vec, vals=10):
    '''
    For use to one-hot encode the 10- possible labels
    '''
#     n = len(vec)
#     out = np.zeros((n, vals))
#     out[range(n), vec] = 1
    return vec

In [60]:
X_train = np.transpose(data["X_train"], [0,2,3,1])[:100]
y_train = one_hot(data["y_train"])[:100]
X_test = np.transpose(data["X_test"], [0,2,3,1])[:100]
y_test = one_hot(data["y_test"])[:100]
X_val = np.transpose(data["X_val"], [0,2,3,1])[:100]
y_val = one_hot(data["y_val"])[:100]

In [61]:
score = model.fc8
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_true, logits = score))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train = optimizer.minimize(cross_entropy)

In [62]:
# Initialize all global variables
init = tf.global_variables_initializer()

In [63]:
epoch = 15
batch_size = 10
N = 100
with tf.Session() as sess:  
    sess.run(init)
    
    for j in range(epoch):
        print("Epoch %d" %(j + 1))
        for i in range(0, N, batch_size):
            # get next batch of data.
            start = i * batch_size
            end = (i+1) * batch_size
            if end > N:
                end = N
            X = X_train[start:end]
            y = y_train[start:end]
            # On training set.
            sess.run(train, feed_dict = {x : X, y_true : y, keep_prob : 0.5})
        
        matches = tf.equal(tf.argmax(score, 1), y_true)
        acc = tf.reduce_mean(tf.cast(matches, tf.float32))
        
        val_accuracy = acc.eval(feed_dict = {x : X_val, y_true : y_val, keep_prob : 1.0})
        
        print("Val Accuracy: %g" %(val_accuracy))

        
        
    test_accuracy = acc.eval(feed_dict = {x : X_test, y_true : y_test, keep_prob : 1.0})

    print("Accuracy: %g" %(test_accuracy))

        

Epoch 1


InvalidArgumentError: logits and labels must be broadcastable: logits_size=[0,10] labels_size=[1,0]
	 [[node softmax_cross_entropy_with_logits (defined at <ipython-input-61-362521d83150>:2) ]]

Original stack trace for 'softmax_cross_entropy_with_logits':
  File "/home/harry/anaconda3/envs/DL_tf_pytorch/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/harry/anaconda3/envs/DL_tf_pytorch/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/harry/anaconda3/envs/DL_tf_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/harry/anaconda3/envs/DL_tf_pytorch/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/harry/anaconda3/envs/DL_tf_pytorch/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/harry/anaconda3/env